# Experiment 005: bbox3 Optimization + Lattice Initialization

Part 1: bbox3 on baseline - NO IMPROVEMENT (confirmed)
Part 2: Lattice initialization for large N + SA optimization

In [ ]:
import os
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
import time
import subprocess

np.random.seed(42)

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def get_tree_polygon(x, y, deg):
    base_poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(base_poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

def get_bounding_box_side(trees):
    if not trees:
        return float('inf')
    all_x, all_y = [], []
    for x, y, deg in trees:
        poly = get_tree_polygon(x, y, deg)
        bounds = poly.bounds
        all_x.extend([bounds[0], bounds[2]])
        all_y.extend([bounds[1], bounds[3]])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [get_tree_polygon(x, y, deg) for x, y, deg in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        candidates = tree_index.query(poly)
        for j in candidates:
            if i != j and poly.intersects(polygons[j]) and not poly.touches(polygons[j]):
                return True
    return False

def recenter_trees(trees):
    if not trees:
        return trees
    all_x, all_y = [], []
    for x, y, deg in trees:
        poly = get_tree_polygon(x, y, deg)
        bounds = poly.bounds
        all_x.extend([bounds[0], bounds[2]])
        all_y.extend([bounds[1], bounds[3]])
    cx = (min(all_x) + max(all_x)) / 2
    cy = (min(all_y) + max(all_y)) / 2
    return [(x - cx, y - cy, deg) for x, y, deg in trees]

print("Functions defined!")

In [ ]:
# Part 1: bbox3 optimization results
print("=== Part 1: bbox3 Optimization ===")
print("bbox3 was run with -n 10000 -r 16 on baseline")
print("Result: NO IMPROVEMENT (score stayed at 70.676102)")
print("This confirms the baseline is at a very strong local optimum.")
print()

# Load baseline
baseline_path = '/home/code/experiments/001_baseline/santa-2025.csv'
df_baseline = pd.read_csv(baseline_path, dtype=str)

def load_all_configs(df):
    configs = {}
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        rows = df[df['id'].str.startswith(prefix)]
        trees = []
        for _, row in rows.iterrows():
            x = float(str(row['x']).replace('s', ''))
            y = float(str(row['y']).replace('s', ''))
            deg = float(str(row['deg']).replace('s', ''))
            trees.append((x, y, deg))
        configs[n] = trees
    return configs

baseline_configs = load_all_configs(df_baseline)
baseline_scores = {n: get_bounding_box_side(baseline_configs[n])**2 / n for n in range(1, 201)}
baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

In [ ]:
# Part 2: Lattice initialization for large N
print("\n=== Part 2: Lattice Initialization ===")

def create_lattice_config(n, nx, ny, dx=0.8, dy=0.8, angle1=0, angle2=180):
    """Create a lattice configuration with 2 trees per cell."""
    trees = []
    for j in range(ny):
        for i in range(nx):
            if len(trees) >= n:
                break
            x1 = i * dx
            y1 = j * dy
            trees.append((x1, y1, angle1))
            if len(trees) < n:
                # Second tree offset within cell
                x2 = x1 + dx * 0.4
                y2 = y1 + dy * 0.4
                trees.append((x2, y2, angle2))
        if len(trees) >= n:
            break
    return trees[:n]

# Target N values for lattice initialization
lattice_targets = [
    (72, 6, 6),    # 6x6 grid = 72 trees (2 per cell)
    (100, 5, 10),  # 5x10 grid = 100 trees
    (144, 6, 12),  # 6x12 grid = 144 trees
    (156, 6, 13),  # 6x13 grid = 156 trees
    (196, 7, 14),  # 7x14 grid = 196 trees
    (200, 10, 10), # 10x10 grid = 200 trees
]

print("Testing lattice configurations...")
for n, nx, ny in lattice_targets:
    # Try different dx, dy values
    best_lattice = None
    best_score = float('inf')
    
    for dx in np.arange(0.6, 1.2, 0.1):
        for dy in np.arange(0.6, 1.2, 0.1):
            for angle1 in [0, 45, 90]:
                for angle2 in [180, 225, 270]:
                    lattice = create_lattice_config(n, nx, ny, dx, dy, angle1, angle2)
                    if len(lattice) == n and not has_overlap(lattice):
                        lattice = recenter_trees(lattice)
                        score = get_bounding_box_side(lattice)**2 / n
                        if score < best_score:
                            best_score = score
                            best_lattice = lattice
    
    baseline_score_n = baseline_scores[n]
    if best_lattice and best_score < baseline_score_n:
        print(f"N={n}: IMPROVED! {baseline_score_n:.6f} -> {best_score:.6f}")
    elif best_lattice:
        print(f"N={n}: No improvement. Baseline: {baseline_score_n:.6f}, Lattice: {best_score:.6f}")
    else:
        print(f"N={n}: No valid lattice found")

In [ ]:
# The lattice approach didn't find improvements either.
# Let's check if the baseline is truly optimal by examining the per-N scores.

print("\n=== Per-N Score Analysis ===")
print("Top 20 N values by score contribution:")
sorted_scores = sorted([(n, baseline_scores[n]) for n in range(1, 201)], key=lambda x: -x[1])
for n, score in sorted_scores[:20]:
    side = get_bounding_box_side(baseline_configs[n])
    print(f"  N={n:3d}: score={score:.6f}, side={side:.6f}")

print(f"\nTotal baseline score: {baseline_total:.6f}")
print(f"Target score: 68.919154")
print(f"Gap: {baseline_total - 68.919154:.6f}")

In [ ]:
# Save the baseline as submission (no improvement found)
submission_rows = []
for n in range(1, 201):
    trees = baseline_configs[n]
    for i, (x, y, deg) in enumerate(trees):
        row_id = f'{n:03d}_{i}'
        submission_rows.append({
            'id': row_id,
            'x': f's{x:.18f}',
            'y': f's{y:.18f}',
            'deg': f's{deg:.18f}'
        })

submission_df = pd.DataFrame(submission_rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('/home/code/experiments/005_bbox3_optimization/submission.csv', index=False)
print(f"Saved submission (baseline, no improvement found)")

print(f"\n=== FINAL SCORE: {baseline_total:.6f} ===")
print(f"=== IMPROVEMENT: 0.000000 ===")